In [4]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    XLMRobertaTokenizer
)

In [37]:
def question_type(data, nama):
    num_apa = 0
    num_dimana = 0
    num_kapan = 0
    num_siapa = 0
    num_bagaimana = 0
    num_kenapa = 0
    num_berapa = 0
    num_others = 0
    
    for i in data['question']:
        current_question = i.split()
        
        if 'Apa' in current_question: num_apa += 1
        elif 'Apakah' in current_question:num_apa += 1
        elif 'apa' in current_question:num_apa += 1
        elif 'apakah' in current_question:num_apa += 1
        
        elif 'Dimana' in current_question: num_dimana += 1
        elif 'dimana' in current_question: num_dimana += 1
        elif 'mana' in current_question: num_dimana += 1
        
        elif 'Kapan' in current_question: num_kapan += 1
        elif 'kapan' in current_question: num_kapan += 1
        
        elif 'Siapa' in current_question: num_siapa += 1
        elif 'siapa' in current_question: num_siapa += 1
        
        elif 'Bagaimana' in current_question: num_bagaimana += 1
        elif 'bagaimana' in current_question: num_bagaimana += 1

        elif 'Mengapa' in current_question: num_kenapa += 1
        elif 'Kenapa' in current_question: num_kenapa += 1
        elif 'mengapa' in current_question: num_kenapa += 1
        elif 'kenapa' in current_question: num_kenapa += 1
        
        elif 'Berapa' in current_question: num_berapa += 1
        elif 'Berapakah' in current_question: num_berapa += 1
        elif 'berapa' in current_question: num_berapa += 1
        elif 'berapakah' in current_question: num_berapa += 1
        
        else: num_others += 1
        
    denominator = len(data['question'])
    num_apa_percentage = num_apa / denominator
    num_dimana_percentage = num_dimana / denominator
    num_kapan_percentage = num_kapan / denominator
    num_siapa_percentage = num_siapa / denominator
    num_kenapa_percentage = num_kenapa / denominator
    num_bagaimana_percentage = num_bagaimana / denominator
    num_berapa_percentage = num_berapa / denominator
    num_others_percentage = num_others / denominator
    
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa}")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan}")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others}")
    print()
        
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others_percentage * 100} %")

In [83]:
def pick_a_row(data, question_type, number):
    
    cols = ['context', 'question', 'answer']
    target_question_type_df = pd.DataFrame(columns=cols)

    for i in range(len(data)):

        current_question = data['question'][i].split()
        
        if question_type == 'lainnya':
            if 'Apa' in current_question: pass
            elif 'Apakah' in current_question: pass
            elif 'apa' in current_question: pass
            elif 'apakah' in current_question: pass
            elif 'Dimana' in current_question: pass
            elif 'dimana' in current_question: pass
            elif 'mana' in current_question: pass
            elif 'Kapan' in current_question: pass
            elif 'kapan' in current_question: pass
            elif 'Siapa' in current_question: pass
            elif 'siapa' in current_question: pass
            elif 'Bagaimana' in current_question: pass
            elif 'bagaimana' in current_question: pass
            elif 'Mengapa' in current_question: pass
            elif 'Kenapa' in current_question: pass
            elif 'mengapa' in current_question: pass
            elif 'kenapa' in current_question: pass
            elif 'Berapa' in current_question: pass
            elif 'Berapakah' in current_question: pass
            elif 'berapa' in current_question: pass
            elif 'berapakah' in current_question: pass

            else: 
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)
            continue

        for j in question_type:
            
            if j in current_question: 
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)

    target_question_type_df = target_question_type_df.sample(n=number)
    return target_question_type_df

# Bagian IDK-MRC

In [73]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    for j in df_test["qas"][i]:
        if len(j['answers']) != 0:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': j['answers'][0]['text'] 
                                                           }, 
                                                           ignore_index=True)
        else:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': "-", 
                                                           }, 
                                                           ignore_index=True)

test_dataset = Dataset.from_dict(new_df_test)

data_qas_id_idk_mrc = DatasetDict({"test": test_dataset})
data_qas_id_idk_mrc = pd.DataFrame(data_qas_id_idk_mrc['test'])
data_qas_id_idk_mrc

  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 378/378 [00:01<00:00, 197.89it/s]


,context,question,answer
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan ?,1959
1,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan komputer mikro mulai ditinggalkan?,-
2,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan Amerik...,-
3,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer makro mulai dikembangkan?,-
4,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,Komomene sebuah nama yang berdasarkan pada nam...
...,...,...,...
839,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,-
840,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, kota ini ..."
841,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa vo...",-
842,"Tanpa beasiswa, Ogilvy tidak bisa kuliah di Fe...",Apa alasan Ogilvy tidak bisa kuliah di Fettes ...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...


In [43]:
question_type(data=data_qas_id_idk_mrc, nama="IDK-MRC Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 14
# DIMANA sebanyak 13
# KAPAN sebanyak 14
# SIAPA sebanyak 13
# KENAPA sebanyak 8
# BAGAIMANA sebanyak 12
# BERAPA sebanyak 13
# LAINNYA sebanyak 13

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 268
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 49
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 173
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 76
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 8
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 12
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 109
IDK-MRC Test memiliki banyak pertanyaan LAINNYA sekitar: 149

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 31.753554502369667 %
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 5.80568720379147 %
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 20.497630331753555 %
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 9.004739336492891 %
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 0.9478672985781991 %
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 1.4218009478672986 %
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 12.914691943

In [90]:
apa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Apa', 'Apakah', 'apa', 'apakah'], number=14)
dimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Dimana', 'dimana', 'mana'], number=13)
kapan_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Kapan', 'kapan'], number=14)
siapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Siapa', 'siapa'], number=13)
kenapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Mengapa', 'Kenapa', 'mengapa', 'kenapa'], number=8)
bagaimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Bagaimana', 'bagaimana'], number=12)
berapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type=['Berapa', 'Berapakah', 'berapa', 'berapakah'], number=13)
lainnya_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="lainnya", number=14)

In [91]:
apa_df_idk_mrc

,context,question,answer
198,Polimer berasal dari 2 kata yaitu poly(banyak)...,apakah yang dimaksud dengan polimer ?,sejumlah molekul besar yang terdiri dari unit ...
244,Antijamur (atau dapat disebut juga antifungal)...,apakah arti dari antifungal?,golongan obat yang bersifat fungisida atau fun...
141,Badan Narkotika Nasional (disingkat BNN) adala...,Apa nama perusahaan pemerintah yang menangani ...,-
184,Simfoni No. 9 dalam tangga nada D minor adalah...,apakah karya terakhir beethoven?,Simfoni No. 9 dalam tangga nada D minor
115,jmpl|200px|Sebuah gereja Kristen berada di bel...,Apa agama mayoritas penduduk korea ?,tidak memiliki agama
102,Kepala Staf TNI Angkatan Darat atau biasa disi...,Apa keuntungan menjadi KASAD?,-
186,Orde Lama dalam sejarah politik Indonesia meru...,Apa sebutan untuk pemerintahan Soekarno yang t...,Orde Lama
248,Penyakit genetik atau kelainan genetik adalah ...,Apa yang dimaksud dengan persamaan genetik?,-
262,"Endokrinologi (dari bahasa Yunani: ἔνδον, endo...",Apa definisi dari endokrinolog?,cabang ilmu kedokteran yang menangani penyakit...
193,Putri Marino () adalah seorang aktris dan pres...,apakah Film terakhir yang pernah diperankan ol...,-


In [92]:
dimana_df_idk_mrc

,context,question,answer
33,"Oleh karena itu, banyak kantor pemerintahan ya...",Di mana letak Balai Kota Inggris?,-
8,"Petrus Lombardus (c. 1096, Novara, Lombardia[1...",Di mana Paulos Lombardus meninggal?,-
1,"Di Hispania, Ataulf dengan tidak hati-hati men...",Dimana Raja Ataulf meninggal?,istana Barcelona
39,"Republik Malta, umumnya dikenal sebagai Malta,...",Dimana letak daerah Malta di Eropa ?,"80km (50mi) di selatan dari Italia, 284km (176..."
0,"Ir. Basuki Tjahaja Purnama, M.M. (EYD: Basuki ...",Di mana Basuki Tjahaja Purnama?,-
26,"Tempat kelahirannya adalah Gorinchem, dan ayah...",Dimana Philip Heinrich Christoph Hofman lahir?,Gorinchem
9,"Insiden Honnoji (本能寺の変, Honnōji-no-hen) adalah...",Dimana Insiden Honnoji terjadi ?,Honnoji
20,Bär dilahirkan tanggal 21 Maret 1913 di Sommer...,Dimana Oskar-Heinz “Pritzl” Bär lahir ?,Sommerfeld dekat Lipsk
36,Aquitaine (bahasa Gascon dan Occitan: Aquitàni...,Dimana letak Aquitania?,Pegunungan Pyrenees
19,Su Dingfang dilahirkan di Wuyi (sekarang Hengs...,Di mana Su Tang lahir?,-


In [93]:
kapan_df_idk_mrc

,context,question,answer
36,"HSBC Holdings PLC (NYSE:, LSE:, SEHK:, Euronex...",Kapan HSBC Holdings PNC ditutup?,-
149,Yoshihiro Francis Fukuyama (lahir 27 Oktober 1...,Kapan Francis Fukuyama lahir ?,27 Oktober 1952
51,Indonesia Raya atau Melayu Raya adalah konsep ...,Kapan konsep Raya dibentuk?,-
155,"Dilahirkan dengan nama Ahmad Syahiduddin, pada...",Kapan Drs. K.H. Ahmad Syahiduddin lahir?,15 Maret 1956
52,Kekaisaran Bulgaria Pertama ([Първo Българско ...,Kapan Kekaisaran Bulgaria Pertama didirikan?,632
124,Thariqah Qadiriyah Naqsabandiyah atau Thoriqoh...,Kapan Tarekat Qadiriyah-NAqsyabandiyah dibubar...,-
154,Pabrik Zippo didirikan pada akhir tahun 1932 o...,Kapan produk pertama Zippo mulai diluncurkan?,Januari dan Februari 1933
113,Setelah resmi menjadi prajurit TNI Angkatan Ud...,Kapan anak dari Mayor PNB Djalaludin Tantu ber...,-
38,"Insiden Honnoji (本能寺の変, Honnōji-no-hen) adalah...",Kapan Insiden Hincroft terjadi?,-
24,Yuchi Jingde lahir tahun 585 di Shuozhou (seka...,Kapan Yuchi Jinde dipenjara?,-


In [94]:
siapa_df_idk_mrc

,context,question,answer
2,Namun berkat bantuan para pengikutnya dan masy...,Siapa penyebab perang Pagung Wilis?,-
47,Eiffel... I'm in Love adalah film drama remaja...,Siapa artis wanita pemeran utama film terbaru?,-
11,"Baudouin adalah putra Eustace II, Comte Boulog...",Siapa anak Baudouin I?,-
42,Jabatan Panglima TNI pertama kali dijabat oleh...,Siapa jenderal terakhir TNI?,-
74,"Pada tahun 1948, saat George Habash masih bela...",Siapa anggota keluarga Habash yang selamat dal...,-
26,"Pada tahun 1978, Gereja Bethany didirikan oleh...",Siapa pendiri Gereja Bethany?,Prof. DR. Abraham Alex Tanuseputra
45,Pemimpin lama Republik Sosialis Soviet Turkmen...,Siapa yang bukan presiden Kazakhstan pertama?,-
31,Lupin III awalnya merupakan sebuah serial mang...,Siapa karakter utama dalam The Castle of Cagli...,Arsène Lupin III
1,Encik Yusof bin Ishak () adalah Presiden Singa...,Siapa wakil presiden pertama Singapura?,-
65,Athanasius Kircher adalah sarjana Yesuit Jerma...,Siapa yang menciptakan alat Megafon?,Athanasius Kircher


In [95]:
kenapa_df_idk_mrc

,context,question,answer
3,Protista adalah mikroorganisme eukariota yang ...,Mengapa protista dikelompokkan ke dalam satu k...,-
6,PT Kharisma Starvision Plus atau dengan nama u...,Mengapa PT. Kharisma Starvision Plus didirikan?,-
4,"Iodin atau Iodium ( - ungu), adalah unsur kimi...",Mengapa nomor atom Iodin 53?,-
1,Hamzah bin Abdul-Muththalib (Arabic: حمزه بن ع...,Kenapa Hamzah bin Abdul-Muththalib tidak dijul...,-
2,Protista adalah mikroorganisme eukariota yang ...,Mengapa protista tidak dikelompokkan ke dalam ...,"Dari sudut pandang taksonomi, pengelompokan in..."
7,"Nepal nyaris berbentuk segi empat, dengan panj...",Mengapa daerah Nepal dan Bangladesh menyatu?,-
5,Penemu lampu lalu lintas adalah Lester Farnswo...,Mengapa sistem sinyal stop dan go dipergunakan...,-
0,Hamzah bin Abdul-Muththalib (Arabic: حمزه بن ع...,Kenapa Hamzah bin Abdul-Muththalib dijuluki si...,karena kepahlawanannya saat membela Islam


In [96]:
bagaimana_df_idk_mrc

,context,question,answer
11,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,-
6,Suaka margasatwa Muara Angke (SMMA) adalah seb...,Bagaimana letak Suaka margasatwa Muara Angke?,-
7,"Dilahirkan dengan nama Ahmad Syahiduddin, pada...",Bagaimana cara Drs. K.H. Ahmad Syahiduddin dil...,-
5,Gerakan Kepanduan adalah sebuah gerakan pembin...,Bagaimana tujuan pendidikan formal kepramukaan...,-
3,Aquitaine (bahasa Gascon dan Occitan: Aquitàni...,Bagaimana dengan letak Austria?,-
8,Vaidišová memulai debutnya dalam tenis profesi...,Bagaimana Nicole Vaidišová mulai menjadi petenis?,-
9,Pemerintah Pakistan (Urdu: حکومتِ پاکستان‎) ad...,Bagaimana sistem pemerintahan Pakistan ?,republik demokrasi parlementer
4,Gerakan Kepanduan adalah sebuah gerakan pembin...,Bagaimana tujuan pendidikan non-formal kepramu...,Tujuan ini dicapai melalui program latihan dan...
1,Istilah Penyiar Radio ini pertama kali digunak...,Bagaimana musik hip hop menjadi bahasa di dunia?,-
2,Para pakar studi Biblika memberikan nama yang ...,Bagaimana Kitab Yesaya ditulis?,-


In [97]:
berapa_df_idk_mrc

,context,question,answer
78,"Secara geografis, Pulau Enggano berada di wila...",Berapakah luas anak Pulau Enggano?,-
92,Berikut daftar kecamatan dan kelurahan di Prov...,berapakah jumlah kecamatan di Nusa Tenggara Ti...,309
96,"Saat ini, Senat Amerika Serikat terdiri dari 1...",berapakah jumlah senator di Amerika Serikat?,100
33,Aleksander Yaneus (Alexander Jannaeus; juga di...,Berapa lama Aleksander Yaneus menjadi raja?,103-76 SM
65,Pengambilan gambar utama pada film dimulai di ...,Tahun berapakah The Greatest Showman batal dir...,-
31,Dua penguasa zaman kuno yang pertama kali meng...,Berapa berat Mesin kepung?,180 ton
40,Abu Dhabi terletak di bagian timur laut Teluk ...,berapakah luas Abu Dhabi?,67.340 kilometer persegi
89,Vaidišová memulai debutnya dalam tenis profesi...,Tahun berapa Nicole Vaidišová berhenti menjadi...,-
75,Penghargaan Pulitzer (bahasa Inggris: Pulitzer...,Berapa banyak penghargaan dalam bidang Jurnali...,-
54,Secara geografis pulau Belitung (Melayu ; Beli...,Berapa luas pulau Karibia?,-


In [98]:
lainnya_df_idk_mrc

,context,question,answer
122,The Rain adalah grup musik Indonesia yang bera...,berapkah jumlah album The Rain?,enam
91,Gedung Kementerian Perindustrian Republik Indo...,Di manakah kantor cabang Kemenperin RI?,-
84,Timur Dekat Kuno merujuk kepada sebuah peradab...,Dimanakah letak Timur Dekat Kuno?,Terdiri dari Mesopotamia (Iraq dan timurlaut S...
72,Kota Merdeka Danzig[1][2][3][4][5][6] (German:...,Di manakah letak Danzig menurut Google Maps?,-
13,Sapta Darma merupakan aliran kerohanian di Ind...,di kota manakah Sapta Darma pertama kali muncul?,Kediri
78,Genosida atau genosid (Bahasa Inggris: genocid...,dari manakah istilah genosida berasal?,bahasa Yunani
128,"Margarita dari Navarra (French: Marguerite, Sp...",Siapakah anak dari Margarita dari Navarra?,Gugghiermu II
133,Kerajaan Bohemia (Czech: České království; Ger...,Dimanakah letak kerajaan Bohemia menurut pemer...,-
15,"Kembali ke Amerika Serikat, Handler mendesain ...",kapankah boneka Barbie pertama kali diciptakan?,9 Maret 1959
5,John Logie Baird () adalah penemu yang pertama...,siapakah penemu televisi?,John Logie Baird


# Bagian TYDIQA-ID

In [27]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'tydiqa_id' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in range(len(df_test['context'])):
    answer_start = df_test['context'][i].index(df_test['label'][i])
    answer_end = answer_start + len(df_test['label'][i])
    new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                        'question': df_test["question"][i], 
                                        'answer': df_test["label"][i]}, ignore_index=True)
   
test_dataset = Dataset.from_dict(new_df_test)
data_qas_id_tydiqa_id = DatasetDict({"test": test_dataset})
data_qas_id_tydiqa_id = pd.DataFrame(data_qas_id_tydiqa_id['test'])
data_qas_id_tydiqa_id

  0%|          | 0/3 [00:00<?, ?it/s]

,context,question,answer
0,Ernest Douwes Dekker wafat dini hari tanggal 2...,dimanakah Dr. Ernest François Eugène Douwes D...,28 Agustus 1950
1,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl
2,Sebagai tindak lanjut Atlantic Charter tersebu...,Kapan PBB mulai terbentuk ?,24 Oktober 1945
3,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,Puri Dragsholm
4,"Lahir di Sheffield, South Yorkshire, Vardy mem...",Dimana Jamie Richard Vardy lahir?,"Sheffield, South Yorkshire"
...,...,...,...
850,Raden Patah (Jawa: ꦫꦢꦺꦤ꧀ꦦꦠꦃcode: jav promoted ...,Siapa raja Demak pertama ?,Raden Patah
851,Laut dalam adalah lapisan terbawah dari lautan...,Berapakah kedalaman laut yang disebut dengan l...,1828 m
852,"Justus Heurnius (lahir di Utrecht, Belanda, 15...",Kapan Justus Heurnius lahir ?,1587
853,Frekuensi suara atau frekuensi audio yaitu get...,Apakah yang dimaksud dengan frekuensi audio?,getaran frekuensi yang terdengar oleh manusia ...


In [42]:
question_type(data=data_qas_id_tydiqa_id, nama="TyDIQA-ID Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 15
# DIMANA sebanyak 14
# KAPAN sebanyak 15
# SIAPA sebanyak 15
# KENAPA sebanyak 6
# BAGAIMANA sebanyak 5
# BERAPA sebanyak 15
# LAINNYA sebanyak 15

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 301
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 34
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 136
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 56
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 6
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 5
TyDIQA-ID Test memiliki banyak pertanyaan BERAPA sekitar: 131
TyDIQA-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 186

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 35.2046783625731 %
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 3.976608187134503 %
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 15.906432748538013 %
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 6.549707602339182 %
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 0.7017543859649122 %
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 0.5847953216374269 %
TyDIQA-ID Test memiliki banyak pertanyaan 

In [89]:
apa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Apa', 'Apakah', 'apa', 'apakah'], number=15)
dimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Dimana', 'dimana', 'mana'], number=14)
kapan_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Kapan', 'kapan'], number=15)
siapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Siapa', 'siapa'], number=15)
kenapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Mengapa', 'Kenapa', 'mengapa', 'kenapa'], number=6)
bagaimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Bagaimana', 'bagaimana'], number=5)
berapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type=['Berapa', 'Berapakah', 'berapa', 'berapakah'], number=15)
lainnya_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="lainnya", number=15)

In [99]:
apa_df_tydiqa_id

,context,question,answer
22,Hermafroditisme tidak terlalu luas dikenal dal...,Hewan apakah yang memiliki Hermafrodit?,siput dan siput bugil
83,150px|jmpl|Urin di dalam toilet di museum seni...,apakah yang dimaksudkan dengan air seni?,cairan sisa yang diekskresikan oleh ginjal yan...
215,Periode Proto Tiga Kerajaan (Masa Sebelum Tiga...,apakah kerajaan terbesar yang pernah ada di Ko...,Dongbuyeo (Buyeo Timur) dan Bukbuyeo (Buyeo Ut...
157,Yuan (dikenal dengan nama Renminbi (RMB) di RR...,Apa mata uang Tiongkok ?,Yuan
61,Kata “Balocci” secara harfiah diduga berasal d...,Apa arti kata Balocci ?,arak kecut
75,"""Punk Rock"" adalah salah satu cabang genre dar...",Apakah band punk rock pertama di Inggris ?,"The Clash, Sex Pistols"
267,Jenis terbesar dari anakonda adalah anakonda h...,Apa jenis ular terbesar di dunia ?,anakonda hijau
68,Air suci adalah air yang telah diberkati dan d...,apa yang dimaksud dengan air suci pada agama k...,air yang telah diberkati dan dipisahkan untuk ...
266,"Pada tahun 1337 Fidiricu meninggal di Paternò,...",apakah penyebab utama kematian Fidiricu II?,1337
12,Diatessaron adalah sebuah karya dari Tatianus ...,Apakah agama Tatianus?,Kristen


In [100]:
dimana_df_tydiqa_id

,context,question,answer
12,Sungai Rhein berasal dari Pegunungan Alpen di ...,Dimana letak anak sungai Vorderrhein?,Danau Tuma dekat celah Oberalp dan melewati Ru...
18,Jam Gadang adalah nama untuk menara jam yang t...,Dimana letak Jam Gadang?,"kota Bukittinggi, Sumatera Barat, Indonesia"
3,Bandar Pasir Mandoge adalah sebuah kecamatan d...,"Dimana letak daerah ""Pardembanan""?","Kabupaten Asahan, Sumatera Utara, Indonesia"
23,Kepulauan Solomon atau Kepulauan Salomo adalah...,Dimana letak Pulau Solomon ?,Samudra Pasifik bagian selatan
25,Ratu Merah tidak menyadari identitas Alice dan...,Dimana penggambaran film Alice in Wonderland ...,Wonderland
8,Pulau Bali adalah bagian dari Kepulauan Sunda ...,Dimana letak geografis Provinsi Bali?,8°25′23″ Lintang Selatan dan 115°14′55″ Bujur ...
22,Etos perusahaan Nike adalah melibatkan dedikas...,"Dimana kantor pusat perusahaan Nike, Inc.?","Beaverton, Oregon"
0,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,Puri Dragsholm
33,Morfem (English: Morpheme) adalah satuan grama...,Dari mana kata morfem berasal ?,English
5,Force India F1 (nama resmi perusahaan: Sahara ...,Dimana Force India F1 didirikan?,India


In [101]:
kapan_df_tydiqa_id

,context,question,answer
4,"Natal (dari bahasa Portugis yang berarti ""kela...",Kapan kelahiran Yesus diperingati ?,25 Desember
7,Rose mulai membuat siniar pada tanggal 24 Juli...,Kapan Robert Kevin Rose mendirikan Revision3 ?,April 2005
68,PlayStation (bahasa Jepang: プレイステーション) adalah ...,Kapan Playstation pertama dirilis ?,3 Desember 1994
133,"Sebelum dikenali umum sebagai Melly Goeslaw, i...",Kapan Meliana Cessy Goeslaw menikah?,1995
61,"Dinasti Qing (Hanzi: 清朝, hanyu pinyin: Qīng Ch...",Kapan Dinasti Qing Manchu berdiri ?,1644
22,Scream 3 merupakan sebuah film horor yang seba...,Kapan serial Scream ketiga dirilis ?,4 Februari 2000
29,Badan Standardisasi Nasional merupakan Lembaga...,Kapan Badan Standardisasi Nasional didirikan?,997
2,"Dalam Operasi hari Kiamat, Divisi 1 Lintas Uda...",Kapan Operasi hari Kiamat terjadi ?,Mei 1945
70,Ban Joakim Gutkeled menculik putra dan ahli wa...,Kapan István V meninggal?,6 Agustus 1272
108,Yahoo! Inc. didirikan oleh Jerry Yang dan Davi...,Kapan Yahoo! Inc. berdiri ?,Januari 1994


In [102]:
siapa_df_tydiqa_id

,context,question,answer
18,Naruto (ナルト) adalah sebuah serial manga karya ...,Siapa penulis Naruto ?,Masashi Kishimoto
4,"Namun, pada tahun 1997 Nicolas Loufrani[2] men...",Siapa yang menciptakan Emosikon?,Nicolas Loufrani
46,Program AdSense didirikan setelah Google menga...,Siapa pendiri AdSense?,Erick Schmidt
55,Raden Patah (Jawa: ꦫꦢꦺꦤ꧀ꦦꦠꦃcode: jav promoted ...,Siapa raja Demak pertama ?,Raden Patah
36,"Di Grand Final, Ryan komika asal Jakarta yang ...",Siapa pemenang SUCI 1 ?,Ryan
48,Mahabharata (Sanskerta: महाभारत) adalah sebuah...,Siapa pencipta wiracarita Mahabharata?,Begawan Byasa
11,Perang Diponegoro yang juga dikenal dengan seb...,Siapa pihak yang menang dalam Perang Diponegoro?,Belanda
40,Adam Smith dikenal luas dengan teori ekonomi '...,Siapa bapak ilmu ekonomi dunia?,Adam Smith
7,"Pada tanggal 28 November 1938, Jiang dan Mao m...",Siapa nama anak Jiang Qing?,Li Na
23,Seorang DJ bernama Alan Freed (alias Moondog) ...,Siapa yang menciptakan musik rock pertama kali ?,Alan Freed


In [103]:
kenapa_df_tydiqa_id

,context,question,answer
1,Sulaa (juga sering disebut dengan Desa Topa) m...,Kenapa Topa disebut dengan Negeri Sejuta Mata ...,karena memiliki sumber mata air alami sebagai ...
0,Aksi tersebut telah digambarkan sebagai mengam...,Mengapa Aksi 9 September 1796 berakhir mengamb...,tidak ada pihak yang bisa mencapai hasil yang ...
2,"25 Januari pagi, keadaan berlangsung seperti b...",Mengapa KMP Tampomas II mengalami kebakaran?,"kebocoran bahan bakar, dan puntung rokok yang ..."
3,Nicola Sacco () dan Bartolomeo Vanzetti () ada...,Mengapa Nicola Sacco dan Bartolomeo Vanzetti d...,melakukan pembunuhan dan pencurian
4,Amelia tampaknya ingin mencoba membuat uang da...,Mengapa Amelia Elizabeth Dyer née Hobley membu...,ingin mencoba membuat uang dari bisnis peterna...
5,Etnografi Dunia Maya adalah adaptasi metode pe...,mengapa kultur dan manusia tak dapat dipisahkan?,Setiap tindakan manusia pasti bermuatan kultur...


In [104]:
bagaimana_df_tydiqa_id

,context,question,answer
1,BJT (Bipolar Junction Transistor) adalah salah...,Bagaimana Cara kerja BJT?,dua diode yang terminal positif atau negatifny...
3,Kerja kursi lontar mengikuti bentuk kokpit pes...,Bagaimana bentuk Kursi lontar?,mengikuti bentuk kokpit pesawat tempur yang di...
2,Pengobatan TB menggunakan antibiotik untuk mem...,Bagaimana cara mengobati Tuberkulosis?,menggunakan antibiotik untuk membunuh bakterinya
0,Pada hewan multiseluler renda kompleks sel pad...,Bagaimana Bryozoa berkembangbiak?,renda kompleks sel pada permukaan individu ind...
4,"Mesir berbentuk republik sejak 18 Juni 1953, M...",Bagaimana sistem pemerintahan di Mesir ?,republik


In [105]:
berapa_df_tydiqa_id

,context,question,answer
116,Nama untuk kawasan ini pertama kali dipakai pa...,Berapa jumlah negara di Asia Tenggara ?,sebelas
78,Restorasi Meiji terjadi pada tahun 1866 sampai...,Berapa lama Restorasi Meiji terjadi?,1866 sampai 1869
84,Keempat aliran tersebut diakui sebagai gaya Ka...,Ada berapakah aliran dalam karate ?,empat
16,Arab Saudi[lower-alpha 1] (/ˌsɔːdiː əˈreɪbiə/(...,berapakah luas Arab Saudi?,"2,150,000km"
27,Sensus Penduduk Indonesia 2010 (disingkat SP20...,Berapa populasi penduduk Indonesia tahun 2010 ?,juta orang na
9,Kabupaten Pesawaran merupakan daerah penyangga...,Berapa luas Kabupaten Pesawaran?,"1.173,77 km2"
11,"Luas Aceh 5.677.081 ha, dengan hutan sebagai l...",berapakah luas Aceh?,5.677.081 ha
82,Kelahiran SWA tidak dapat dipisahkan dari sala...,tahun berapakah SWA didirikan?,1985
22,Kompleks bangunan Candi Penataran menempati ar...,berapakah luas Candi Penataran?,12.946 meter persegi
47,Burj Khalifa (bahasa Arab برج خليفة yang berar...,Berapa ketinggian Burj Khalifa?,828 meter


In [106]:
lainnya_df_tydiqa_id

,context,question,answer
156,Federasi Malaysia adalah sebuah monarki konsti...,Bagaimanakah sistem pemerintahan Malaysia ?,monarki konstitusional
129,Pertempuran memperebutkan Lapangan Udara Hende...,Dimanakah letak Lapangan Udara Henderson ?,sekitar Guadalkanal pada Kepulauan Solomon
68,Ketua Umum: Megawati Soekarnoputri Ketua Dewan...,Siapakah ketua umum PDI tahun 2018 ?,Megawati Soekarnoputri
97,Tembok Berlin (German: Berliner Mauer) adalah ...,Kapankah Tembok Berlin dibangun ?,13 Agustus 1961
31,Film Hachikō Monogatari karya sutradara Seijir...,siapakah sutradara Hachiko: A Dog's Story?,Lasse Hallström
171,"Dinamai sesuai nama perancangnya, Gustave Eiff...",Siapakah nama arsitek Eiffel?,Gustave Eiffel
20,Tentara Sukarela Pembela Tanah Air atau PETA(郷...,Kapankah PETA berdiri ?,3 Oktober 1943
117,Garry Kasparov lahir dengan nama Gari Weinstei...,Dari manakah Garry Kasparov berasal ?,"Baku, Azerbaijan"
1,John Fitzgerald Kennedy lahir di 83 Beals Stre...,siapakah orang tua John Fitzgerald Kennedy?,"Joseph Patrick ""Joe"" Kennedy, Sr. (1888–1969) ..."
119,Teman-temannya menyarankan dia untuk memperlam...,dimanakah Ibnu Sina dimakamkan?,"Hamadan, Iran"


# Bagian SQUAD-ID

In [26]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': eval(df_validation["answer"][i][0])['text']},
                                ignore_index=True)

validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id_squad_id = DatasetDict({"validation": validation_dataset})
data_qas_id_squad_id = pd.DataFrame(data_qas_id_squad_id['validation'])
data_qas_id_squad_id

  0%|          | 0/2 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████| 11873/11873 [00:28<00:00, 415.34it/s]


,context,question,answer
0,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Di negara apa Normandia berada?,Perancis
1,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Kapan Normandia di Normandia?,10 dan ke
2,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Dari negara mana asal Norse?,"Denmark, Islandia dan Norwegia"
3,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Siapa pemimpin Norse?,Rollo
4,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Abad berapa pertama kali Normandia mendapatkan...,abad ke-10
...,...,...,...
11868,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapa satuan gaya yang jarang digunakan sama ...,sthène
11869,"Gaya pon memiliki padanan metrik, yang lebih j...",Apa yang tidak memiliki mitra metrik?,pound-force
11870,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapakah gaya yang diberikan oleh gravitasi s...,kilogram-force
11871,"Gaya pon memiliki padanan metrik, yang lebih j...",Kekuatan apa yang mengarah ke satuan massa yan...,kilogram-force


In [40]:
question_type(data=data_qas_id_squad_id, nama="SQuAD-ID Test")

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 6215
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 899
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 692
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 1030
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 191
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 340
SQuAD-ID Test memiliki banyak pertanyaan BERAPA sekitar: 1470
SQuAD-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 1036

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 52.345658216120604 %
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 7.5718015665796345 %
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 5.82835003790112 %
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 8.67514528762739 %
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 1.6086919902299335 %
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 2.863640192032342 %
SQuAD-ID Test memiliki banyak pertanyaan BERAPA

In [31]:
data_qas_id_idk_mrc.to_csv("idk_mrc.csv")
data_qas_id_tydiqa_id.to_csv("tydiqa_id.csv")
#data_qas_id_squad_id.to_excel("squad_id.xlsx")